In [15]:
import numpy as np
import pandas as pd
from numpy.random import randn

import matplotlib.pyplot as plt
import seaborn as sns
import dataprep as dp
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor

from sklearn.model_selection import GridSearchCV, cross_val_score, KFold, train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, LogisticRegression, RANSACRegressor, Ridge, Lasso, ElasticNet

import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import GridSearchCV, cross_val_score, KFold
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler, MaxAbsScaler
from sklearn.model_selection import cross_val_score,KFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.metrics import accuracy_score, f1_score, confusion_matrix, r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression, RANSACRegressor, ElasticNet
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, AdaBoostRegressor
from sklearn.svm import LinearSVC
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor

In [5]:
train_df = pd.read_csv('train.csv')
test_df= pd.read_csv('test.csv')
submission= pd.read_csv('submission.csv')

In [6]:
columns_train=['hour', 'hour_bef_temperature', 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility', 'hour_bef_ozone','count']
columns_test=['hour', 'hour_bef_temperature', 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility', 'hour_bef_ozone']
train=train_df[columns_train]
test=test_df[columns_test]
train.drop(index=[934,1035], inplace=True)

In [1]:
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

it_train = train.copy()

it_train = IterativeImputer(random_state=2021).fit_transform(it_train)

itImp = pd.DataFrame(it_train)
itImp.columns = train.columns

itImp.isna().sum()

NameError: name 'train' is not defined

In [8]:
it_test = test.copy()

it_test = IterativeImputer(random_state=2021).fit_transform(it_test)

itImp1 = pd.DataFrame(it_test)
itImp1.columns = test.columns

itImp1.isna().sum()

hour                    0
hour_bef_temperature    0
hour_bef_windspeed      0
hour_bef_humidity       0
hour_bef_visibility     0
hour_bef_ozone          0
dtype: int64

In [12]:
itImp_mid = itImp.copy()
col_name=['hour', 'hour_bef_temperature', 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_visibility', 'hour_bef_ozone']
for ilt in col_name:
    Q1=itImp_mid[ilt].quantile(0.25)
    Q3=itImp_mid[ilt].quantile(0.75)
    IQR=Q3-Q1
    train_delout=itImp_mid[(itImp_mid[ilt]<(Q1 - 1.5*IQR)) | (itImp_mid[ilt]>(Q3+1.5*IQR))]
    itImp_mid=itImp_mid.drop(train_delout.index, axis=0)
itImp_mid[col_name].info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1433 entries, 0 to 1456
Data columns (total 6 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   hour                  1433 non-null   float64
 1   hour_bef_temperature  1433 non-null   float64
 2   hour_bef_windspeed    1433 non-null   float64
 3   hour_bef_humidity     1433 non-null   float64
 4   hour_bef_visibility   1433 non-null   float64
 5   hour_bef_ozone        1433 non-null   float64
dtypes: float64(6)
memory usage: 78.4 KB


In [13]:
col = ['hour', 'hour_bef_temperature', 'hour_bef_windspeed', 'hour_bef_humidity', 'hour_bef_ozone']

X_train = itImp_mid[col]
y_train = itImp_mid[['count']]
X_test = itImp1[col]

In [ ]:
eT_reg=ExtraTreesRegressor()
n_estimator_param=[i for i in range(100,1000,100)]
max_depth_param=[i for i in range(5,20)]
min_samples_split_param=[i for i in range(2,10)]

param_grid={'n_estimators' : n_estimator_param,
             'max_depth': max_depth_param,
            'min_samples_split': min_samples_split_param}

gs= GridSearchCV(eT_reg, param_grid=param_grid, scoring='neg_mean_squared_error',cv=5,n_jobs=-1)
gs=gs.fit(X_train, y_train)
print(gs.best_params_)

In [14]:
from sklearn.ensemble import ExtraTreesRegressor

best_reg = ExtraTreesRegressor(n_estimators=400, max_depth=10, random_state=2021)
best_reg.fit(X_train, y_train)
y_preds = best_reg.predict(X_test)

submission['count'] = y_preds
submission.to_csv('Extra.csv', index = False)